# 1. Формулировка задачи

| Характеристика | Описание |
| --- | --- |
| **Суть задачи** | Агент получает матрицу смежности взвешенного графа, начальную и конечную вершины. Ему нужно найти кратчайший путь. |
| **Регулировка сложности** | Параметр `difficulty` (от 1 до 10) маппится на гиперпараметры графа: количество вершин (например, `N = difficulty * 2 + 3`) и плотность ребер. |
| **Верификация (Verifier)** | Классический алгоритм Дейкстры рассчитывает эталонный кратчайший путь и его стоимость. Ответ агента парсится, суммируются веса ребер предложенного маршрута, и проверяется, совпадает ли эта сумма с минимальной, а также существует ли такой путь в принципе. |

# 2. Импорты

In [1]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; _numpy = f'numpy=={numpy.__version__}'; _pil = f'pillow=={PIL.__version__}'
    except: _numpy = "numpy"; _pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    _vllm, _triton = ('vllm==0.9.2', 'triton==3.2.0') if is_t4 else ('vllm==0.15.1', 'triton')
    !uv pip install -qqq --upgrade {_vllm} {_numpy} {_pil} torchvision bitsandbytes xformers unsloth
    !uv pip install -qqq {_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

In [3]:
%cd /content
!rm -rf HW-2_env
!git clone https://github.com/TebelevGt/HW-2_env.git
%cd HW-2_env/rl-shortest-path-agent

/content
Cloning into 'HW-2_env'...
remote: Enumerating objects: 416, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 416 (delta 27), reused 50 (delta 24), pack-reused 354 (from 1)
Receiving objects: 100% (416/416), 6.24 MiB | 8.54 MiB/s, done.
Resolving deltas: 100% (193/193), done.
/content/HW-2_env/rl-shortest-path-agent


In [4]:
from envs import PathEnv, PathVerifier, ShortestPathDataset, get_shortest_path_dataset
from envs import ModelWrapper, evaluate_agent
import os
import re

In [5]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

# Увеличиваем контекст, так как reasoning-цепочки могут быть длинными
max_seq_length = 4096*2
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-1.5B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Чуть снизил для запаса под генерацию (rollouts)
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 128,     # Рекомендуется: alpha = rank * 2
    lora_dropout = 0.05,  # Добавляем регуляризацию
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,    # Включаем Rank-Stabilized LoRA для лучшей сходимости
)

# ВАЖНО: Для Instruct моделей нужно убедиться, что токенизатор настроен корректно
# Unsloth обычно делает это сам, но для генерации (inference) в RL полезно явно задать:
tokenizer.padding_side = "left" # Для генерации обычно left, для обучения right (Unsloth сам меняет при SFT, но для RL важно проверить)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 02-22 16:05:56 [__init__.py:244] Automatically detected platform cuda.
ERROR 02-22 16:05:59 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-22 16:06:11 [vllm_utils.py:723] Unsloth: Patching vLLM v1 graph capture
INFO 02-22 16:06:11 [vllm_utils.py:752] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2026.2.1: Fast Qwen2 patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM l

`torch_dtype` is deprecated! Use `dtype` instead!


WARNING 02-22 16:06:33 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 02-22 16:06:33 [config.py:1472] Using max model len 8192
WARNING 02-22 16:06:34 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 02-22 16:06:35 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=4096.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection', 'model.layers.0.self_attn', 'model.layers.1.mlp', 'model.layers.2.mlp', 'model.layers.3.mlp', 'model.layers.7.mlp', 'model.layers.24.mlp', 'model.layers.26.mlp', 'model.layers.15.self_attn'], 'llm_int8_threshold': 6.0}
INFO 02-22 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.2.1 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [6]:
train_dataset=get_shortest_path_dataset('data/train_v1.pkl')
train_dataset[0]

{'prompt': [{'content': '\nYou are a pathfinding expert. Find a valid path between the given start and end nodes.\nFirst, explore the graph step-by-step inside <reasoning> tags.\nThen, output ONLY the final sequence of node indices, comma-separated, inside <answer> tags.\n\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n',
   'role': 'system'},
  {'content': 'You are an expert at graph pathfinding. Find a valid path from Node 4 to Node 1 in the graph below.\n\nGraph connections:\nNode 0 is connected to: Node 3 (weight 6), Node 7 (weight 2), Node 8 (weight 2), Node 9 (weight 7), Node 10 (weight 2)\nNode 1 is connected to: Node 2 (weight 6), Node 3 (weight 6), Node 8 (weight 10), Node 11 (weight 5)\nNode 2 is connected to: Node 1 (weight 6), Node 3 (weight 1), Node 7 (weight 8), Node 11 (weight 9)\nNode 3 is connected to: Node 0 (weight 6), Node 1 (weight 6), Node 2 (weight 1), Node 4 (weight 2), Node 5 (weight 7), Node 7 (weight 2), Node 9 (w

In [ ]:
from trl import GRPOTrainer,GRPOConfig
from envs import  correctness_reward_func, reasoning_length_reward_func, format_reward_func

training_args = GRPOConfig(
    use_vllm=True,  # use vllm for fast inference!
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,  # Increase to 4 for smoother training
    num_generations=8,  # Decrease if out of memory
    max_prompt_length=256,
    max_completion_length=200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps=250,
    save_steps=250,
    max_grad_norm=0.1,
    report_to="none",  # Can use Weights & Biases
    output_dir="outputs",
)

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        correctness_reward_func,
        reasoning_length_reward_func,
        format_reward_func
    ],
    args=training_args,
    train_dataset=get_shortest_path_dataset('data/train_v1.pkl')
)

trainer.train()

Unsloth: We now expect `per_device_train_batch_size` * `gradient_accumulation_steps` * `world_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 73,859,072 of 1,617,573,376 (4.57% trained)


INFO 02-22 16:07:24 [peft_helper.py:58] Loading LoRA weights trained with rsLoRA.
Unsloth: Will smartly offload gradients to save VRAM!

[Train] Start: 2 | End: 3 | Ideal: 2, 8, 3 (Cost: 11) | Agent: Empty/Parse Error | Reward: 0.0

[Train] Start: 2 | End: 3
  Ideal Path: 2, 8, 3 (Cost: 11)
  Agent Path: [2, 3, 9, 11, 4, 7]
  Agent Cost: 0 (Broken)
  Exists in Graph: False | Reached End: False
  Reward: 0.3000

[Train] Start: 2 | End: 3 | Ideal: 2, 8, 3 (Cost: 11) | Agent: Empty/Parse Error | Reward: 0.0

[Train] Start: 2 | End: 3 | Ideal: 2, 8, 3 (Cost: 11) | Agent: Empty/Parse Error | Reward: 0.0

[Train] Start: 2 | End: 3 | Ideal: 2, 8, 3 (Cost: 11) | Agent: Empty/Parse Error | Reward: 0.0

[Train] Start: 2 | End: 3
  Ideal Path: 2, 8, 3 (Cost: 11)
  Agent Path: [2, 10, 9]
  Agent Cost: 0 (Broken)
  Exists in Graph: False | Reached End: False
  Reward: 0.3000

[Train] Start: 2 | End: 3 | Ideal: 2, 8, 3 (Cost: 11) | Agent: Empty/Parse Error | Reward: 0.0

[Train] Start: 2 | End: 3
  

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std,rewards / reasoning_length_reward_func / mean,rewards / reasoning_length_reward_func / std,rewards / format_reward_func / mean,rewards / format_reward_func / std
1,0.299400,0.340417,0.413157,156.750000,65.000000,200.000000,0.625000,84.666672,65.000000,102.000000,0.000000,0.112500,0.155265,0.059167,0.111455,0.168750,0.203430
2,0.111600,0.231250,0.367363,176.625000,98.000000,200.000000,0.625000,137.666672,98.000000,161.000000,0.000000,0.018750,0.053033,0.062500,0.176777,0.150000,0.212132
3,0.191600,0.187500,0.407738,172.000000,85.000000,200.000000,0.625000,125.333336,85.000000,186.000000,0.000080,0.037500,0.106066,0.043750,0.123744,0.106250,0.200780
4,0.224400,0.581250,1.644023,183.375000,67.000000,200.000000,0.875000,67.000000,67.000000,67.000000,0.009342,0.537500,1.520280,0.000000,0.000000,0.043750,0.123744
5,0.224300,0.632500,0.554862,155.875000,62.000000,200.000000,0.500000,111.750000,62.000000,147.000000,0.031846,0.131250,0.148655,0.232500,0.218795,0.268750,0.252045
6,0.205700,0.421875,0.586011,170.875000,118.000000,200.000000,0.625000,122.333336,118.000000,126.000000,0.237617,0.093750,0.137419,0.140625,0.195115,0.187500,0.258775
7,0.162500,0.865417,0.552610,151.500000,90.000000,200.000000,0.125000,144.571442,90.000000,197.000000,2.916405,0.150000,0.160357,0.340417,0.221943,0.375000,0.231455
8,-0.035500,1.638333,1.482320,124.750000,69.000000,192.000000,0.000000,124.750000,69.000000,192.000000,0.006561,0.750000,1.452830,0.388333,0.109265,0.500000,0.000000
9,-0.055500,1.543333,1.481981,101.500000,47.000000,195.000000,0.000000,101.500000,47.000000,195.000000,0.015566,0.725000,1.450862,0.318333,0.150177,0.500000,0.000000
10,0.062000,2.998750,2.376368,128.125000,32.000000,200.000000,0.250000,104.166672,32.000000,157.000000,0.035399,2.225000,2.221165,0.380000,0.164681,0.393750,0.159099


Выходные данные были обрезаны до нескольких последних строк (5000).
  Agent Path: [7, 3]
  Agent Cost: 7
  Exists in Graph: True | Reached End: False
  Reward: 0.5500

[Train] Start: 8 | End: 2
  Ideal Path: 8, 2 (Cost: 5)
  Agent Path: [8, 5, 2]
  Agent Cost: 17
  Exists in Graph: True | Reached End: True
  Reward: 4.3000

[Train] Start: 8 | End: 2
  Ideal Path: 8, 2 (Cost: 5)
  Agent Path: [8, 2]
  Agent Cost: 5
  Exists in Graph: True | Reached End: True
  Reward: 4.3000

[Train] Start: 8 | End: 2
  Ideal Path: 8, 2 (Cost: 5)
  Agent Path: [8, 2]
  Agent Cost: 5
  Exists in Graph: True | Reached End: True
  Reward: 4.3000

[Train] Start: 8 | End: 2
  Ideal Path: 8, 2 (Cost: 5)
  Agent Path: [8, 2]
  Agent Cost: 5
  Exists in Graph: True | Reached End: True
  Reward: 4.3000

[Train] Start: 8 | End: 2
  Ideal Path: 8, 2 (Cost: 5)
  Agent Path: [8, 10, 5, 2]
  Agent Cost: 0 (Broken)
  Exists in Graph: False | Reached End: True
  Reward: 0.3000

[Train] Start: 8 | End: 2
  Ideal Path: 8

In [2]:
from envs import ModelWrapper
agent = ModelWrapper(trainer.model, tokenizer)

# 3. Сохраняем обертку в .pkl (через torch.save, так как там тензоры)
torch.save(agent, "outputs/grpo_agent.pkl")

ModuleNotFoundError: No module named 'envs'